# sxs_catalog_download_example

This notebook demonstrates how to use the `sxs` python library to download data from the SXS Catalog of waveforms hosted on Zenodo (https://github.com/moble/sxs). The catalog is available at https://black-holes.org/waveforms and is described in https://arxiv.org/abs/1904.04831.

You can install `sxs` via `pip install sxs`. You'll need version `2019.4.15.16.32.10` or later.

Note: You should use a recent version of python 3, such as 3.6.5, to run this notebook. I installed python 3.6.5 using anaconda (https://www.anaconda.com). 

Note: I found I had to also install tqdm, which you can install similarly. This should be installed when you install the `sxs` library with `pip`, but in case you see errors about being unable to import `tqdm`, `pip install tqdm` solves this.

## How to download data

This section demonstrates how to download simulation data from Zenodo. You can download data from one simulation or from multiple simulations at once.

In [ ]:
# Uncomment the next line if running on Microsoft Azure / Google Colaboratory
#!pip install sxs

In [ ]:
# For downloading data
import sxs
from sxs import zenodo as zen

# For interacting with the data
import h5py
import numpy as np
from matplotlib import pyplot as plt
import json

The next two cells download two simulations: 
- SXS:BBH:0305, created to model the first gravitational-wave observation, GW150914 (https://doi.org/10.1103/PhysRevLett.116.061102)
- SXS:BBH:0317, created to model the second gravitational-wave observation, GW151226 (https://doi.org/10.1103/PhysRevLett.116.241103)

The cells download metadata and data for the black hole horizons at multiple resolutions and the gravitational waves at the highest resolution

In [ ]:
zen.download.matching("common-metadata.txt", "metadata.txt", "metadata.json", \
                      "Horizons.h5", \
                      sxs_ids = ['SXS:BBH:0305', 'SXS:BBH:0317'], \
                      dry_run = False, \
                      highest_lev_only = False)

In [ ]:
zen.download.matching("rhOverM_Asymptotic_GeometricUnits_CoM.h5", \
                      sxs_ids = ['SXS:BBH:0305', 'SXS:BBH:0317'], \
                      dry_run = False, \
                      highest_lev_only = True)

## Read in the downloaded data

Read in metadata (a set of a few numbers characterizing the simulation), horizon data (such as how the black holes move and how their masses change), and gravitational-wave data (how much the waves would stretch and squeeze our detectors.

In [ ]:
horizons_low_res = {}
horizons_low_res["SXS:BBH:0305"] = h5py.File("SXS_BBH_0305/Lev2/Horizons.h5", 'r')
horizons_low_res["SXS:BBH:0317"] = h5py.File("SXS_BBH_0317/Lev1/Horizons.h5", 'r')

horizons_medium_res = {}
horizons_medium_res["SXS:BBH:0305"] = h5py.File("SXS_BBH_0305/Lev4/Horizons.h5", 'r')
horizons_medium_res["SXS:BBH:0317"] = h5py.File("SXS_BBH_0317/Lev2/Horizons.h5", 'r')

horizons_high_res = {}
horizons_high_res["SXS:BBH:0305"] = h5py.File("SXS_BBH_0305/Lev6/Horizons.h5", 'r')
horizons_high_res["SXS:BBH:0317"] = h5py.File("SXS_BBH_0317/Lev3/Horizons.h5", 'r')

gw = {}
gw["SXS:BBH:0305"] = h5py.File("SXS_BBH_0305/rhOverM_Asymptotic_GeometricUnits_CoM.h5", 'r')
gw["SXS:BBH:0317"] = h5py.File("SXS_BBH_0317/rhOverM_Asymptotic_GeometricUnits_CoM.h5", 'r')

metadata_low_res = {}
with open("SXS_BBH_0305/Lev2/metadata.json") as file:
    metadata_low_res["SXS:BBH:0305"] = json.load(file)
with open("SXS_BBH_0317/Lev1/metadata.json") as file:
    metadata_low_res["SXS:BBH:0317"] = json.load(file)
    
metadata_medium_res = {}
with open("SXS_BBH_0305/Lev4/metadata.json") as file:
    metadata_medium_res["SXS:BBH:0305"] = json.load(file)
with open("SXS_BBH_0317/Lev2/metadata.json") as file:
    metadata_medium_res["SXS:BBH:0317"] = json.load(file)
    
metadata_high_res = {}
with open("SXS_BBH_0305/Lev6/metadata.json") as file:
    metadata_high_res["SXS:BBH:0305"] = json.load(file)
with open("SXS_BBH_0317/Lev3/metadata.json") as file:
    metadata_high_res["SXS:BBH:0317"] = json.load(file)

# Choose total mass and distance to binary

Here, for each observation, choose how far away (in meters) the wave is from Earth, and the sum of the two black hole masses (in solar masses).

In [ ]:
total_mass = {}
total_mass["SXS:BBH:0305"] = 66.2 # m_sun
total_mass["SXS:BBH:0317"] = 21.4 # m_sun

distance = {}
distance["SXS:BBH:0305"] = 1.3e25 # meters
distance["SXS:BBH:0317"] = 1.3e25 # meters

Goverc2m = 1.47708885e3 # meters / solar mass
Goverc3s = 4.92703806e-6 # seconds / solar mass

# Plot the black-hole masses

In [ ]:
sxs_bbh_0305_mass1 = horizons_high_res["SXS:BBH:0305"]["AhA.dir"]["ChristodoulouMass.dat"]
sxs_bbh_0305_mass2 = horizons_high_res["SXS:BBH:0305"]["AhB.dir"]["ChristodoulouMass.dat"]
sxs_bbh_0305_massFinal = horizons_high_res["SXS:BBH:0305"]["AhC.dir"]["ChristodoulouMass.dat"]

plt.clf()
plt.plot(Goverc3s * total_mass["SXS:BBH:0305"] * sxs_bbh_0305_mass1[:,0], total_mass["SXS:BBH:0305"] * sxs_bbh_0305_mass1[:,1], label='Mass 1')
plt.plot(Goverc3s * total_mass["SXS:BBH:0305"] * sxs_bbh_0305_mass2[:,0], total_mass["SXS:BBH:0305"] * sxs_bbh_0305_mass2[:,1], label='Mass 2')
plt.plot(Goverc3s * total_mass["SXS:BBH:0305"] * sxs_bbh_0305_massFinal[:,0], total_mass["SXS:BBH:0305"] * sxs_bbh_0305_massFinal[:,1], label='Final mass')
plt.legend(loc='best')
plt.xlabel("Time (seconds)")
plt.ylabel("Mass (solar masses)")
plt.show()

In [ ]:
sxs_bbh_0317_mass1 = horizons_high_res["SXS:BBH:0317"]["AhA.dir"]["ChristodoulouMass.dat"]
sxs_bbh_0317_mass2 = horizons_high_res["SXS:BBH:0317"]["AhB.dir"]["ChristodoulouMass.dat"]
sxs_bbh_0317_massFinal = horizons_high_res["SXS:BBH:0317"]["AhC.dir"]["ChristodoulouMass.dat"]

plt.clf()
plt.plot(Goverc3s * total_mass["SXS:BBH:0317"] * sxs_bbh_0317_mass1[:,0], total_mass["SXS:BBH:0317"] * sxs_bbh_0317_mass1[:,1], label='Mass 1')
plt.plot(Goverc3s * total_mass["SXS:BBH:0317"] * sxs_bbh_0317_mass2[:,0], total_mass["SXS:BBH:0317"] * sxs_bbh_0317_mass2[:,1], label='Mass 2')
plt.plot(Goverc3s * total_mass["SXS:BBH:0317"] * sxs_bbh_0317_massFinal[:,0], total_mass["SXS:BBH:0317"] * sxs_bbh_0317_massFinal[:,1], label='Final mass')
plt.legend(loc='best')
plt.xlabel("Time (seconds)")
plt.ylabel("Mass (solar masses)")
plt.show()

Challenge: add a curve for the total initial mass (sum of mass1 and mass 2)

# Plot black hole orbits

In [ ]:
sxs_bbh_0305_orbit1 = horizons_high_res["SXS:BBH:0305"]["AhA.dir"]["CoordCenterInertial.dat"]
sxs_bbh_0305_orbit2 = horizons_high_res["SXS:BBH:0305"]["AhB.dir"]["CoordCenterInertial.dat"]
sxs_bbh_0305_orbitFinal = horizons_high_res["SXS:BBH:0305"]["AhC.dir"]["CoordCenterInertial.dat"]

plt.clf()
plt.plot(1e-3 * Goverc2m * total_mass["SXS:BBH:0305"] * sxs_bbh_0305_orbit1[:,1], 1e-3 * Goverc2m * total_mass["SXS:BBH:0305"] * sxs_bbh_0305_orbit1[:,2], label='Mass 1')
plt.plot(1e-3 * Goverc2m * total_mass["SXS:BBH:0305"] * sxs_bbh_0305_orbit2[:,1], 1e-3 * Goverc2m * total_mass["SXS:BBH:0305"] * sxs_bbh_0305_orbit2[:,2], label='Mass 2')
plt.legend(loc='lower left')
plt.xlabel("x (km)")
plt.ylabel("y (km)")

plt.gca().set_aspect(1)
plt.title("SXS:BBH:0305 (GW150914)")
plt.show()

In [ ]:
sxs_bbh_0317_orbit1 = horizons_high_res["SXS:BBH:0317"]["AhA.dir"]["CoordCenterInertial.dat"]
sxs_bbh_0317_orbit2 = horizons_high_res["SXS:BBH:0317"]["AhB.dir"]["CoordCenterInertial.dat"]
sxs_bbh_0317_orbitFinal = horizons_high_res["SXS:BBH:0317"]["AhC.dir"]["CoordCenterInertial.dat"]

plt.clf()
plt.plot(1e-3 * Goverc2m * total_mass["SXS:BBH:0317"] * sxs_bbh_0317_orbit1[:,1], 1e-3 * Goverc2m * total_mass["SXS:BBH:0317"] * sxs_bbh_0317_orbit1[:,2], label='Mass 1')
plt.plot(1e-3 * Goverc2m * total_mass["SXS:BBH:0317"] * sxs_bbh_0317_orbit2[:,1], 1e-3 * Goverc2m * total_mass["SXS:BBH:0317"] * sxs_bbh_0317_orbit2[:,2], label='Mass 2')
plt.legend(loc='lower left')
plt.xlabel("x (km)")
plt.ylabel("y (km)")
plt.title("SXS:BBH:0317 (GW151226)")

plt.gca().set_aspect(1)
plt.show()

# Plot gravitational waves

In [ ]:
gw_sxs_bbh_0305 = gw["SXS:BBH:0305"]["Extrapolated_N2.dir"]["Y_l2_m2.dat"]
gw_sxs_bbh_0317 = gw["SXS:BBH:0317"]["Extrapolated_N2.dir"]["Y_l2_m2.dat"]

In [ ]:
plt.clf()
plt.plot(Goverc3s * total_mass["SXS:BBH:0305"] * gw_sxs_bbh_0305[:,0], 
         1.0e21 * Goverc2m * (total_mass["SXS:BBH:0305"] / distance["SXS:BBH:0305"]) * gw_sxs_bbh_0305[:,1])
plt.xlabel("Time (s)")
plt.ylabel("Detector strain (10^-21)")
plt.title("SXS:BBH:0305 (GW150914)")

plt.show()

In [ ]:
plt.clf()
plt.plot(Goverc3s * total_mass["SXS:BBH:0317"] * gw_sxs_bbh_0317[:,0], 
         1.0e21 * Goverc2m * (total_mass["SXS:BBH:0317"] / distance["SXS:BBH:0317"]) * gw_sxs_bbh_0317[:,1])
plt.xlabel("Time (s)")
plt.ylabel("Detector strain (10^-21)")
plt.title("SXS:BBH:0317 (GW151226)")

plt.show()

# Compare initial and final masses at different resolutions

In [ ]:
sxs_bbh_0305_init_mass_low = total_mass["SXS:BBH:0305"] * (metadata_low_res["SXS:BBH:0305"]["reference_mass1"] + metadata_low_res["SXS:BBH:0305"]["reference_mass2"])
sxs_bbh_0305_init_mass_medium = total_mass["SXS:BBH:0305"] * (metadata_medium_res["SXS:BBH:0305"]["reference_mass1"] + metadata_medium_res["SXS:BBH:0305"]["reference_mass2"])
sxs_bbh_0305_init_mass_high = total_mass["SXS:BBH:0305"] * (metadata_high_res["SXS:BBH:0305"]["reference_mass1"] + metadata_high_res["SXS:BBH:0305"]["reference_mass2"])

sxs_bbh_0305_final_mass_low = total_mass["SXS:BBH:0305"] * (metadata_low_res["SXS:BBH:0305"]["remnant_mass"])
sxs_bbh_0305_final_mass_medium = total_mass["SXS:BBH:0305"] * (metadata_medium_res["SXS:BBH:0305"]["remnant_mass"])
sxs_bbh_0305_final_mass_high = total_mass["SXS:BBH:0305"] * (metadata_high_res["SXS:BBH:0305"]["remnant_mass"])

print("SXS:BBH:0305 (GW150914)\n")

print("Low resolution: Initial mass (solar masses): " + str(sxs_bbh_0305_init_mass_low))
print("Medium resolution: Initial mass (solar masses): " + str(sxs_bbh_0305_init_mass_medium))
print("High resolution: Initial mass (solar masses): " + str(sxs_bbh_0305_init_mass_high))

print (" ")

print("Low resolution: Initial mass (solar masses): " + str(sxs_bbh_0305_final_mass_low))
print("Medium resolution: Initial mass (solar masses): " + str(sxs_bbh_0305_final_mass_medium))
print("High resolution: Initial mass (solar masses): " + str(sxs_bbh_0305_final_mass_high))

In [ ]:
sxs_bbh_0317_init_mass_low = total_mass["SXS:BBH:0317"] * (metadata_low_res["SXS:BBH:0305"]["reference_mass1"] + metadata_low_res["SXS:BBH:0305"]["reference_mass2"])
sxs_bbh_0317_init_mass_medium = total_mass["SXS:BBH:0317"] * (metadata_medium_res["SXS:BBH:0305"]["reference_mass1"] + metadata_medium_res["SXS:BBH:0305"]["reference_mass2"])
sxs_bbh_0317_init_mass_high = total_mass["SXS:BBH:0317"] * (metadata_high_res["SXS:BBH:0305"]["reference_mass1"] + metadata_high_res["SXS:BBH:0305"]["reference_mass2"])

sxs_bbh_0317_final_mass_low = total_mass["SXS:BBH:0317"] * (metadata_low_res["SXS:BBH:0317"]["remnant_mass"])
sxs_bbh_0317_final_mass_medium = total_mass["SXS:BBH:0317"] * (metadata_medium_res["SXS:BBH:0317"]["remnant_mass"])
sxs_bbh_0317_final_mass_high = total_mass["SXS:BBH:0317"] * (metadata_high_res["SXS:BBH:0317"]["remnant_mass"])

print("SXS:BBH:0317 (GW151226)\n")

print("Low resolution: Initial mass (solar masses): " + str(sxs_bbh_0317_init_mass_low))
print("Medium resolution: Initial mass (solar masses): " + str(sxs_bbh_0317_init_mass_medium))
print("High resolution: Initial mass (solar masses): " + str(sxs_bbh_0317_init_mass_high))

print (" ")

print("Low resolution: Initial mass (solar masses): " + str(sxs_bbh_0317_final_mass_low))
print("Medium resolution: Initial mass (solar masses): " + str(sxs_bbh_0317_final_mass_medium))
print("High resolution: Initial mass (solar masses): " + str(sxs_bbh_0317_final_mass_high))

# Make sound for gravitational waves

In [ ]:
t305 = Goverc3s * total_mass["SXS:BBH:0305"] * gw_sxs_bbh_0305[:,0]
h305 = 1.0e21 * Goverc2m * (total_mass["SXS:BBH:0305"] / distance["SXS:BBH:0305"]) * gw_sxs_bbh_0305[:,1]
maxh305 = max(h305)

t317 = Goverc3s * total_mass["SXS:BBH:0317"] * gw_sxs_bbh_0317[:,0]
h317 = 1.0e21 * Goverc2m * (total_mass["SXS:BBH:0317"] / distance["SXS:BBH:0317"]) * gw_sxs_bbh_0317[:,1]
maxh317 = max(h317)

In [ ]:
from scipy.interpolate import interp1d

sample_rate = 1e4 #Hz
h305interp = interp1d(t305, h305)(np.arange(t305[0], t305[-1], 1.0 / sample_rate))
h317interp = interp1d(t317, h317)(np.arange(t317[0], t317[-1], 1.0 / sample_rate))

from scipy.io import wavfile
wavfile.write("SXS_BBH_0305.wav", int(sample_rate), h305interp / maxh305)
wavfile.write("SXS_BBH_0317.wav", int(sample_rate), h317interp / maxh317)